In [69]:
import folium
import pandas as pd
from string import Template
from IPython.display import display, HTML

In [70]:
df = pd.read_csv("munro_bagger - Sheet1 (2).csv").fillna("").astype(str)

In [71]:
for col in ["Metres", "Scott", "Matthew"]:
    df[col] = df[col].apply(lambda x: x.split(".")[0])

In [112]:
table_template = Template("""<!DOCTYPE html>
                            <html lang="en">
                            
                            
                            <style>
                              table {
                            
                                    border-collapse: collapse;
                                    font-family: Arial, sans-serif;
                                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                                    background: #fff;
                                  }

                                    th {
                                        border: 1px solid #ddd;
                                        background-color: #f7f7f7;
                                        padding: 12px 15px;
                                        text-align: left;
                                        font-weight: 600;
                                      }
                            
                                td {
                                    border: 1px solid #ddd;
                                    padding: 10px 15px;
                                    text-align: left;
                                  }

                            tbody tr:hover {
                                            background-color: #f1f1f1;
                                          }
                            </style>

                            <script>
                            function filterByColumnCellIds() {
                              const filter1 = document.getElementById("filterCol1").value;
                              const filter2 = document.getElementById("filterCol2").value;
                            
                              const rows = document.querySelectorAll("#mountainTable tbody tr");
                            
                              rows.forEach(row => {
                                const cell1 = row.cells[2]; 
                                const cell2 = row.cells[3]; 
                            
                                const matchCol1 = !filter1 || cell1.id === filter1;
                                const matchCol2 = !filter2 || cell2.id === filter2;
                            
                                row.style.display = (matchCol1 && matchCol2) ? "" : "none";
                              });
                            }   
                            </script>

                                  <script>
                          document.addEventListener("DOMContentLoaded", function () {
                            const searchInput = document.getElementById('searchInput');
                            const rows = document.querySelectorAll('#mountainTable tbody tr');
                        
                            searchInput.addEventListener('input', function () {
                              const filter = searchInput.value.toLowerCase();
                              rows.forEach(row => {
                                const nameText = row.cells[0].textContent.toLowerCase();
                                row.style.display = nameText.includes(filter) ? '' : 'none';
                              });
                            });
                          });
                        </script>

                        
                    

                        <body>

                        <div class="table-content" id="tableContent">
                        <table id="mountainTable">
                        <colgroup>
                        <col style="width: 100px">
                        <col style="width: 80px">
                        <col style="width: 120px">
                        <col style="width: 120px">
                        </colgroup>
                            <thead>
                                <tr>
                                    <th>Name <br> <input type="text" id="searchInput" placeholder="🔍" style="width: 100px"></th>
                                    <th>Height</th>
                                    <th>Matthew
                                    <select id="filterCol1" onchange="filterByColumnCellIds()">
                                      <option value="">All</option>
                                      <option value="bag">Bags</option>
                                      <option value="gap">Gaps</option>
                                    </select>
                                    </th>

                                    <th>Scott
                                    <select id="filterCol2" onchange="filterByColumnCellIds()">
                                      <option value="">All</option>
                                      <option value="bag">Bags</option>
                                      <option value="gap">Gaps</option>
                                    </select>
                                    </th>
                                </tr>
                            </thead>
                            <tbody>
                                $rows
                            </tbody>
                        </table>
                         </div>
                        </body>
                        </html>
                        """)

In [113]:
row_template = Template(
                        """<tr>
                            <td><button onclick="window.map.setView([$lat, $long], 13)">$name</button></td>
                            <td>$height</td>
                            <td id = "$matthew_status"> $matthew</td>
                            <td id = "$scott_status"> $scott</td>
                          </tr>
                          """)

In [114]:
rows_html = ""

for _, row in df.iterrows():

    if row["Matthew"] != "":
        matthew = '<img src="images/matthew.png" width="64" height="80">'
        matthew_status = "bag"
    else:
        matthew = '<img src="images/sad_face.png" width="80" height="80">'
        matthew_status = "gap"
        
    if row["Scott"] != "":
        scott = '<img src="images/scott.png" width="64" height="80">'
        scott_status = "bag"
    else:
        scott = '<img src="images/sad_face.png" width="80" height="80">'
        scott_status = "gap"

    row = row_template.substitute({'name': row["Name"],
                            'lat': row["Latitude"],
                            'long': row["Longitude"],
                            'height': row["Metres"]+"m",
                            'matthew': matthew,
                            'matthew_status': matthew_status,
                            'scott': scott,
                            'scott_status': scott_status,
                            })


    rows_html += row

In [115]:
table_html = table_template.substitute({"rows": rows_html})

In [116]:
with open("test.html", "w", encoding="utf-8") as file:
    file.write(table_html)

In [25]:
x = """
<script>
function filterTableByRowId() {
  const selectedId = document.getElementById("rowFilter").value;
  const rows = document.querySelectorAll("#myTable tbody tr");

  rows.forEach(row => {
    row.style.display = (!selectedId || row.id === selectedId) ? "" : "none";
  });
}
</script>

<select id="rowFilter" onchange="filterTableByRowId()">
  <option value="">Show All</option>
  <option value="matthew_no_bag">Bags</option>
  <option value="matthew_bag">Gaps</option>
</select>

<!-- Table -->
<table id="myTable" border="1">
  <thead>
    <tr><th>Name</th><th>Category</th></tr>
  </thead>
  <tbody>
    <tr id="matthew_no_bag"><td>Alpha</td><td>Category A</td></tr>
    <tr id="matthew_no_bag"><td>Beta</td><td>Category B</td></tr>
    <tr id="matthew_bag"><td>Gamma</td><td>Category C</td></tr>
  </tbody>
</table>

"""

In [26]:
with open("x.html", "w", encoding="utf-8") as file:
    file.write(x)

In [18]:
scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6)

# Updated CSS and JavaScript for the new layout
map_init_js = """
<style>
/* Container for the entire layout */
.layout-container {
  display: flex;
  flex-direction: column;
  height: 100vh;
  font-family: Arial, sans-serif;
}

/* Table panel styling */
.table-panel {
  background: white;
  border-bottom: 2px solid #ccc;
  box-shadow: 0 2px 5px rgba(0,0,0,0.1);
  z-index: 1000;
  position: relative;
}

/* Panel header with collapse button */
.panel-header {
  background-color: #f8f9fa;
  padding: 15px 20px;
  border-bottom: 1px solid #dee2e6;
  display: flex;
  justify-content: space-between;
  align-items: center;
  cursor: pointer;
  user-select: none;
}

.panel-title {
  font-size: 1.2em;
  font-weight: bold;
  margin: 0;
}

.collapse-btn {
  background: none;
  border: none;
  font-size: 1.5em;
  cursor: pointer;
  color: #6c757d;
  transition: transform 0.3s ease;
}

.collapse-btn:hover {
  color: #495057;
}

.collapse-btn.collapsed {
  transform: rotate(180deg);
}

/* Table content area */
.table-content {
  padding: 20px;
  max-height: 400px;
  overflow-y: auto;
  transition: all 0.3s ease;
  opacity: 1;
}

.table-content.collapsed {
  max-height: 0 !important;
  padding-top: 0 !important;
  padding-bottom: 0 !important;
  overflow: hidden !important;
  opacity: 0;
}

/* Search input styling */
#searchInput {
  width: 100%;
  max-width: 400px;
  padding: 10px 15px;
  margin-bottom: 15px;
  font-size: 1em;
  border: 2px solid #e9ecef;
  border-radius: 6px;
  transition: border-color 0.3s ease;
}

#searchInput:focus {
  outline: none;
  border-color: #007bff;
}

/* Table styling */
#mountainTable {
  width: 100%;
  border-collapse: collapse;
  background: white;
  border-radius: 8px;
  overflow: hidden;
  box-shadow: 0 2px 10px rgba(0,0,0,0.1);
}

#mountainTable th {
  background-color: #007bff;
  color: white;
  padding: 15px;
  text-align: left;
  font-weight: bold;
  position: sticky;
  top: 0;
  z-index: 10;
}

#mountainTable td {
  padding: 12px 15px;
  border-bottom: 1px solid #e9ecef;
  vertical-align: middle;
}

#mountainTable tr:hover {
  background-color: #f8f9fa;
}

#mountainTable tr:last-child td {
  border-bottom: none;
}

/* Image styling */
.uniform-img, td img {
  width: 60px;
  height: 60px;
  object-fit: cover;
  border-radius: 6px;
  margin-right: 10px;
  border: 2px solid #e9ecef;
}

/* Map panel styling */
.map-panel {
  flex: 1;
  position: relative;
  min-height: 400px;
}

/* Responsive design */
@media (max-width: 768px) {
  .panel-header {
    padding: 12px 15px;
  }
  
  .panel-title {
    font-size: 1.1em;
  }
  
  .table-content {
    padding: 15px;
    max-height: 300px;
  }
  
  #searchInput {
    max-width: 100%;
  }
  
  #mountainTable th,
  #mountainTable td {
    padding: 10px 8px;
    font-size: 0.9em;
  }
  
  .uniform-img, td img {
    width: 50px;
    height: 50px;
  }
}

@media (max-width: 480px) {
  .panel-header {
    padding: 10px 12px;
  }
  
  .table-content {
    padding: 12px;
  }
  
  #mountainTable th,
  #mountainTable td {
    padding: 8px 6px;
    font-size: 0.8em;
  }
  
  .uniform-img, td img {
    width: 40px;
    height: 40px;
  }
}
</style>

<script>
document.addEventListener("DOMContentLoaded", function() {
    // Find the map object
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
    
    // Collapse/expand functionality with better error handling
    function setupCollapse() {
        const panelHeader = document.getElementById('panelHeader');
        const tableContent = document.getElementById('tableContent');
        const collapseBtn = document.getElementById('collapseBtn');
        
        if (panelHeader && tableContent && collapseBtn) {
            console.log('Setting up collapse functionality');
            
            panelHeader.addEventListener('click', function(e) {
                console.log('Panel header clicked');
                e.preventDefault();
                
                const isCollapsed = tableContent.classList.contains('collapsed');
                
                if (isCollapsed) {
                    // Expand
                    tableContent.classList.remove('collapsed');
                    collapseBtn.innerHTML = '▼';
                    collapseBtn.classList.remove('collapsed');
                } else {
                    // Collapse
                    tableContent.classList.add('collapsed');
                    collapseBtn.innerHTML = '▲';
                    collapseBtn.classList.add('collapsed');
                }
                
                // Trigger map resize after animation completes
                setTimeout(function() {
                    if (window.map) {
                        console.log('Invalidating map size');
                        window.map.invalidateSize();
                    }
                }, 350);
            });
            
            // Also add click handler directly to button
            collapseBtn.addEventListener('click', function(e) {
                e.stopPropagation(); // Prevent double-firing
            });
        } else {
            console.log('Collapse elements not found, retrying...');
            setTimeout(setupCollapse, 100);
        }
    }
    
    setupCollapse();
    
    // Search functionality
    const searchInput = document.getElementById('searchInput');
    const table = document.getElementById('mountainTable');
    const rows = table.querySelectorAll('tbody tr');
    
    searchInput.addEventListener('input', function () {
        const filter = searchInput.value.toLowerCase();
        rows.forEach(row => {
            const nameCell = row.querySelector('td').innerText.toLowerCase();
            row.style.display = nameCell.includes(filter) ? '' : 'none';
        });
    });
});
</script>
"""

# Updated HTML template for the new layout
panel_html_template = Template("""
<div class="layout-container">
    <!-- Table Panel (Top) -->
    <div class="table-panel">
        <div class="panel-header" id="panelHeader">
            <h3 class="panel-title">Munros Data Table</h3>
            <button class="collapse-btn" id="collapseBtn">▼</button>
        </div>
        <div class="table-content" id="tableContent">
            <input type="text" id="searchInput" placeholder="🔍 Search munros...">
            <table id="mountainTable">
                <thead>
                    <tr>
                        <th>Name</th>
                        <th>Height</th>
                        <th>Bags</th>
                    </tr>
                </thead>
                <tbody>
                    $rows
                </tbody>
            </table>
        </div>
    </div>
    
    <!-- Map Panel (Bottom) -->
    <div class="map-panel" id="mapPanel">
        <!-- Map will be rendered here -->
    </div>
</div>
""")

# Generate the panel HTML (you'll need to define rows_html)
panel_html = panel_html_template.substitute({"rows": rows_html})

# Inject the styles and structure
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

# Additional JavaScript to move the map into the map panel
additional_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    // Wait a bit for folium to initialize
    setTimeout(function() {
        const mapElement = document.querySelector('.folium-map');
        const mapPanel = document.getElementById('mapPanel');
        
        if (mapElement && mapPanel) {
            // Move the map into our map panel
            mapPanel.appendChild(mapElement);
            
            // Ensure map fills the panel
            mapElement.style.width = '100%';
            mapElement.style.height = '100%';
            
            // Force map to resize
            if (window.map) {
                window.map.invalidateSize();
            }
        }
    }, 100);
});
</script>
"""

# Add the additional JavaScript
m.get_root().html.add_child(folium.Element(additional_js))

# Generate the panel HTML (you'll need to define rows_html)
panel_html = panel_html_template.substitute({"rows": rows_html})

# Inject the styles and structure
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

# Additional JavaScript to move the map into the map panel
additional_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    // Wait a bit for folium to initialize
    setTimeout(function() {
        const mapElement = document.querySelector('.folium-map');
        const mapPanel = document.getElementById('mapPanel');
        
        if (mapElement && mapPanel) {
            // Move the map into our map panel
            mapPanel.appendChild(mapElement);
            
            // Ensure map fills the panel
            mapElement.style.width = '100%';
            mapElement.style.height = '100%';
            
            // Force map to resize
            if (window.map) {
                window.map.invalidateSize();
            }
        }
    }, 100);
});
</script>
"""

# Add the additional JavaScript
m.get_root().html.add_child(folium.Element(additional_js))

In [19]:
panel_html = panel_html_template.substitute({"rows": rows_html})

# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

In [20]:
matthew_bag = folium.CustomIcon(icon_image="images/matthew.png",
                            icon_size=(16, 16),
                            icon_anchor=(15, 16)
                            )

scott_bag = folium.CustomIcon(icon_image="images/scott.png",
                            icon_size=(16, 16),
                            icon_anchor=(1, 16)
                            )

peak = folium.CustomIcon(icon_image="images/peak.png",
                            icon_size=(16, 16),
                            icon_anchor=(8, 8)
                            )

In [21]:
bag_icons = {"Matthew": matthew_bag,
             "Scott": scott_bag,
            }

In [22]:
def add_peak(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]

    label = "<br>".join([name, f"height: {height}m"])

    icon = peak

    folium.Marker(
                location = coords,
                tooltip = label,
                icon = icon
                ).add_to(m)

    return

In [23]:
def add_bag(row):

    year = row[person]
    icon = bag_icons[person]
    coords = [row["Latitude"], row["Longitude"]]
    
    folium.Marker(
    location = coords,
    tooltip = year,
    icon = icon
    ).add_to(m)

    return

In [24]:
for _, row in df.iterrows():
    add_peak(row)

In [25]:
for person in ["Matthew", "Scott"]:
    
    for _, row in df[df[person] != ""].iterrows():
        add_bag(row)

In [26]:
# Save it
m.save("index.html")

In [ ]:
scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6)

# JavaScript to extract the internal map object and assign it to `window.map`
map_init_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
});
</script>
"""

# HTML control panel
panel_html = """
<div id="custom-controls" style="
    position: absolute;
    top: 10px;
    left: 10px;
    z-index: 9999;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 0 5px rgba(0,0,0,0.3);
    font-family: sans-serif;
">
    <h4>The Board:</h4>



    
    #<button onclick="window.map.setView([40.7128, -74.0060], 12)">New York</button><br><br>
    #<button onclick="window.map.setView([34.0522, -118.2437], 12)">Los Angeles</button><br><br>
    #<button onclick="window.map.setView([51.5074, -0.1278], 12)">London</button>
</div>
"""

# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))




# Save it
m.save("map_with_controls2.html")


In [ ]:
"""<table border="1" cellpadding="8" cellspacing="0">
  <thead>
    <tr>
      <th>Name</th>
      <th>Height</th>
      <th>Bags</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><button onclick="window.map.setView([56.796891, -5.003675], 12)">Ben Nevis [Beinn Nibheis]</button></td>
      <td>1344m</td>
     <td><img src="Scott.png" alt="Description" width="100"></td>
    </tr>
    <tr>
      <td><button onclick="window.map.setView([58.413113, -4.60786], 12)">Ben Hope</button></td>
      <td>927m</td>
     <td></td>
    </tr>
    <tr>
      <td><button onclick="window.map.setView([56.666942, -4.10023], 12)">Schiehallion</button></td>
      <td>1083</td>
     <td><img src="Scott.png" alt="Description" width="100"><img src="Matthew.png" alt="Description" width="100"></td>
    </tr>
  </tbody>
</table>"""

In [ ]:
<td><button onclick="window.map.setView([58.413113, -4.60786], 12)">Ben Hope</button></td>
<td><button onclick="window.map.setView([56.666942, -4.10023], 12)">Schiehallion</button></td>

In [ ]:
matthew

In [4]:
scotland_center = [56.4907, -4.2026]

# Create the map centered on Scotland
m = folium.Map(location=scotland_center, zoom_start=6)

In [5]:
bag_icon = folium.CustomIcon(icon_image="bag_icon.png",
                            icon_size=(16, 16),
                            icon_anchor=(8, 8)
                            )

no_bag_icon = folium.CustomIcon(icon_image="gap_icon.png",
                            icon_size=(14, 14),
                            icon_anchor=(7, 7)
                            )

In [6]:
def add_marker(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]
    year = row["Matthew"]

    if year != "":
        icon = bag_icon
        label = "<br>".join([name, f"height: {height}m", f"bagged: {year}"])
    else:
        icon = no_bag_icon
        label = "<br>".join([name, f"height: {height}m"])
        
    folium.Marker(
    location = coords,
    tooltip = label,
    icon = icon
    ).add_to(m)

    return

In [7]:
for _, row in df.head(10).iterrows():
    add_marker(row)

In [8]:
m.get_root().html.add_child(folium.Element("""
<script>
    // Expose the map after it's created
    window.addEventListener('load', () => {
        window.myMap = map;

        // Listen for messages from parent
        window.addEventListener('message', (event) => {
            try {
                const data = JSON.parse(event.data);
                if (data.type === 'goto') {
                    map.setView([data.lat, data.lng], data.zoom);
                }
            } catch (e) {
                console.error('Invalid message:', e);
            }
        });
    });
</script>
"""))

m.save("map.html")

In [9]:
m.save("index.html")

In [11]:
df

,Name,Region,County,Metres,Feet,Latitude,Longitude,Matthew
0,Ben Nevis [Beinn Nibheis],04A: Fort William to Loch Treig & Loch Leven,Highland,1344,4411,56.796891,-5.003675,
1,Ben Macdui [Beinn Macduibh],08A: Cairngorms,Aberdeenshire/Moray,1309,4296,57.070368,-3.669099,2024
2,Braeriach,08A: Cairngorms,Aberdeenshire/Highland,1296,4252,57.078298,-3.728373,
3,Cairn Toul,08A: Cairngorms,Aberdeenshire,1291,4236,57.054406,-3.710757,
4,Sgor an Lochain Uaine,08A: Cairngorms,Aberdeenshire,1258,4127,57.058376,-3.725897,
...,...,...,...,...,...,...,...,...
277,Meall na Teanga,10C: Loch Arkaig to Glen Moriston,Highland,916,3008,56.98903,-4.93094,
278,Beinn a' Chleibh,01D: Inveraray to Crianlarich,Argyll and Bute/Stirling,916,3006,56.39023,-4.83563,
279,Ben Vane,01D: Inveraray to Crianlarich,Argyll and Bute,915,3004,56.249786,-4.781655,
280,Carn Aosda,06B: Pitlochry to Braemar & Blairgowrie,Aberdeenshire,915,3003,56.895696,-3.423274,2024


In [ ]:
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Folium Map in Panel</title>
  <style>
    html, body {
      margin: 0;
      height: 100%;
      font-family: sans-serif;
    }

    .container {
      display: flex;
      height: 100%;
    }

    .panel {
      width: 300px;
      background: #f8f8f8;
      padding: 1em;
      box-shadow: 2px 0 5px rgba(0,0,0,0.1);
      overflow-y: auto;
    }

    .map-frame {
      flex: 1;
      border: none;
    }
  </style>
</head>
<body>
  <div class="container">
    <div class="panel">
      <h2>Info Panel</h2>
      <p>This panel can contain filters, stats, or controls.</p>
    </div>
    <iframe src="index.html" class="map-frame"></iframe>
  </div>
</body>
</html>


In [13]:
import folium

# Create map
m = folium.Map(location=[40, -100], zoom_start=4)

# JavaScript to extract the internal map object and assign it to `window.map`
map_init_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
});
</script>
"""

# HTML control panel
panel_html = """
<div id="custom-controls" style="
    position: absolute;
    top: 10px;
    left: 10px;
    z-index: 9999;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 0 5px rgba(0,0,0,0.3);
    font-family: sans-serif;
">
    <h4>Go To:</h4>
    <button onclick="window.map.setView([40.7128, -74.0060], 12)">New York</button><br><br>
    <button onclick="window.map.setView([34.0522, -118.2437], 12)">Los Angeles</button><br><br>
    <button onclick="window.map.setView([51.5074, -0.1278], 12)">London</button>
</div>
"""

# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

# Save it
m.save("map_with_controls2.html")
